In [ ]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 100
import plotly.express as px


# Find impact of an unsub. Portfolios include: aax, dup, firefly, roaming, renewal

In [ ]:
%%bigquery post_unsub_price_plan_change_df



WITH unsubscribe_msisdn_date AS (
SELECT  
  BAN,
  MSISDN,
  MAX(dt) AS unsub_date
FROM `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.post_mob_cust_price_plan_change_marketing_event_ts` 
WHERE UNSUBSCRIBE = 1
GROUP BY BAN, MSISDN
)

SELECT 
  A.*  
FROM `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.post_mob_cust_price_plan_change_marketing_event_ts` A 
INNER JOIN unsubscribe_msisdn_date B 
ON A.BAN = B.BAN AND A.MSISDN = B.MSISDN 
WHERE 
  A.dt >= B.unsub_date
  AND A.marketing_event = 0 -- natural price plan changes only 
  AND A.pp_recur_chrg_amt != 0 -- I believe when pp_recur_chrg_amt becomes 0, it basically means they stopped their plan
ORDER BY BAN, MSISDN, dt 


Following data is interpreted as. For all customers that have unsubscribed from a marketing campaign, this is the average of all price plan changes that occured after their unsub in 2022. This can be thought of as organic changes

In [ ]:
post_unsub_price_plan_change_df.change_in_pp_recur_chrg_amt.describe()

Find change in price plan amount after they have converted. Note because I am doing a group by, it will not consider instances where customers convert multiple times but rather only the most recent conversion

In [ ]:
%%bigquery post_convert_price_plan_change_df

WITH unsubscribe_msisdn AS (
SELECT  
  distinct MSISDN as unsub_msisdn,
FROM `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.post_mob_cust_price_plan_change_marketing_event_ts` 
WHERE UNSUBSCRIBE = 1

),

customers_with_conversion AS (
  SELECT 
    BAN,
    MSISDN,
    MAX(dt) AS convert_date
  FROM `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.post_mob_cust_price_plan_change_marketing_event_ts` 
  WHERE CONVERSION = 1 AND MSISDN NOT IN (SELECT unsub_msisdn FROM unsubscribe_msisdn)
  GROUP BY BAN, MSISDN
)

SELECT
  A.*
FROM `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.post_mob_cust_price_plan_change_dup_event_ts` A 
INNER JOIN customers_with_conversion B
ON A.BAN = B.BAN AND A.MSISDN = B.MSISDN
WHERE
  A.dt >= B.convert_date AND 
  A.marketing_event = 0 
  AND A.pp_recur_chrg_amt != 0 

In [ ]:
post_convert_price_plan_change_df.change_in_pp_recur_chrg_amt.describe()

Below number is subtracting the mean of the post conversion change in price plan price with post unsub change in price plan. Can we say each unsub costs TELUS 1.45 * 24  =  $34.8 per customer. <br>

For each campaign, we could then perform the calculation on number of unsubs saved of target over control 

In [ ]:
post_convert_price_plan_change_df.change_in_pp_recur_chrg_amt.mean() - post_unsub_price_plan_change_df.change_in_pp_recur_chrg_amt.mean()

In [ ]:
1.45 * 24

This value can be used as a starting guide when we begin to optimize for record allocation. Ie. If a campaign has record allocation for 200k customers but our model only identifies 100k as high propensity, we save 100k. Knowing this portfolios past campaign's performance, we have a historical unsub rate to work with. Using the value calculated above, by not targeting 100k customers, we prevent 100k customers * campaign historical unsub rate customers from unsubscribing, which costs TELUS $34.8 over 24 months per customer

In [ ]:
100000 * 0.01 * 34.8

Depending on the record allocation and number of saves, we should be able to generate 5 figure savings per campaign based on this proposed method

# Count of Change in Price plan by Month

In [ ]:
%%bigquery price_plan_change_by_month_cnt

SELECT 
  EXTRACT(MONTH FROM dt) as month_,
  ROUND(change_in_pp_recur_chrg_amt/5)*5 as change_in_pp_recur_chrg_amt,
  count(*) AS count_
FROM  `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.post_mob_cust_price_plan_change_marketing_event_ts`
WHERE marketing_event = 0
AND ROUND(change_in_pp_recur_chrg_amt/5)*5 != 0
GROUP BY 1,2
ORDER BY 1 ASC 

In [ ]:
price_plan_change_by_month_cnt.change_in_pp_recur_chrg_amt.nunique()

In [ ]:
price_plan_change_by_month_cnt

In [ ]:
fig = px.line(price_plan_change_by_month_cnt.loc[(price_plan_change_by_month_cnt.change_in_pp_recur_chrg_amt > 0) & 
                                                 (price_plan_change_by_month_cnt.month_ > 1)],
              x='month_', y='count_', color='change_in_pp_recur_chrg_amt')
fig.show()            

In [ ]:
fig = px.line(price_plan_change_by_month_cnt.loc[(price_plan_change_by_month_cnt.change_in_pp_recur_chrg_amt < 0) & 
                                                 (price_plan_change_by_month_cnt.month_ > 1)],
              x='month_', y='count_', color='change_in_pp_recur_chrg_amt')
fig.show()            

In [ ]:
%%bigquery price_plan_change_cnt

SELECT 
  change_in_pp_recur_chrg_amt,
  count(*) AS count_
FROM  `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.post_mob_cust_price_plan_change_marketing_event_ts`
WHERE marketing_event = 0 AND change_in_pp_recur_chrg_amt != 0
GROUP BY 1
ORDER BY 1 ASC 

In [ ]:

fig = px.line(price_plan_change_cnt,
              x='change_in_pp_recur_chrg_amt', y='count_')
fig.show()            